# 爬下来的数据的存储和读取

## 回顾
- 爬取帖子列表页
- 爬取帖子详情页
- [新]存储帖子列表页的信息（标题、URL、作者、时间、阅读量）
- [新]对每一条帖子存储内容、所有评论

## 新知识
- 使用 pandas 操作 CSV
- 使用正则表达式进行内容匹配

In [1]:
!pip install pandas

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
# import 基础包
import requests
from bs4 import BeautifulSoup

# 公共 header，可以参见 postman 生成的 Python 代码，这个通常每次使用的时候都是固定的
headers = {
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8,zh-TW;q=0.7,fr;q=0.6',
  'Cache-Control': 'max-age=0',
  'Connection': 'keep-alive',
  'Cookie': 'll="108288"; bid=EUIXgN0ZKpc; _pk_id.100001.8cb4=1f708fde13a7265b.1700533653.; __utmc=30149280; viewed="36165055"; douban-fav-remind=1; __yadk_uid=8ArcxBZuzhGsH9PICN0uVjxPJzsUPXf0; _ga=GA1.2.1030669451.1701136885; _gid=GA1.2.1551607325.1701136885; _ga_Y4GN1R87RG=GS1.1.1701136884.1.0.1701136900.0.0.0; __utmz=30149280.1701150588.3.2.utmcsr=localhost:8888|utmccn=(referral)|utmcmd=referral|utmcct=/; ap_v=0,6.0; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1701180456%2C%22http%3A%2F%2Flocalhost%3A8888%2F%22%5D; _pk_ses.100001.8cb4=1; __utma=30149280.1324323786.1700533655.1701175760.1701180456.5; __utmt=1; __utmb=30149280.12.9.1701181143253; bid=qYTbTuF2dfs',
  'Referer': 'https://www.douban.com/group/596337/discussion?start=0&type=new',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-User': '?1',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
  'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"'
}
payload={}

# 列表页面函数

In [3]:
# 初始化一个新的 DataFrame 并指定列名
import pandas as pd
def query_page(group, start: int):
    url = "https://www.douban.com/group/{}/discussion?start={}&type=new".format(group,start)
    response = requests.request("GET", url, headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    # arr = soup.select("td.title > a")
    # times = soup.select("td.time")
    arr = soup.select("table.olt > tr:not(.th)")
    discussions = pd.DataFrame(columns=['url', 'title', 'author', 'author_id', 'reply_count', 'time'])
    for item in arr:
        tds = item.find_all('td')
        href = tds[0].find("a")['href']
        title = tds[0].find("a")['title']
        author = tds[1].text.replace("\n", "")
        author_id = tds[1].find("a")['href'].replace('https://www.douban.com/people/', '').replace('/', '')
        reply_count = tds[2].text
        tm = tds[3].text
        # 向 Dataframe 追加行
        row = pd.DataFrame({
                'url': href,
                'title': title,
                'author': author, 
                'author_id': author_id,
                'reply_count': reply_count,
                'time': tm,
            }, index=[0])
        discussions = pd.concat([discussions, row], ignore_index=True)
    return discussions
pd.concat([query_page(596337, 0),query_page(596337, 30)] , ignore_index=True)

,url,title,author,author_id,reply_count,time
0,https://www.douban.com/group/topic/298659522/,酱香米线,momo,87792824,,12-03 21:35
1,https://www.douban.com/group/topic/298659471/,自己做的饭！！,芒果,fhim333,,12-03 21:35
2,https://www.douban.com/group/topic/298642292/,求助｜放在冷藏里的生饺子是冷水下锅还是热水下锅呀,无酒令游,171280252,16,12-03 21:34
3,https://www.douban.com/group/topic/298648738/,用盐杀了，也用厨房用纸吸了水，炒出来还是很水,ihgh,epicromance,19,12-03 21:26
4,https://www.douban.com/group/topic/298648014/,馒头二次醒发够软蒸出来为什么硬？,柳柳,148106748,20,12-03 21:23
5,https://www.douban.com/group/topic/298658556/,有个问题请教一下,十二楼五城,163324605,,12-03 21:20
6,https://www.douban.com/group/topic/298632568/,有朋友认得这种蔬菜叫什么名字吗，今天买菜第一次见,e,275632699,38,12-03 21:10
7,https://www.douban.com/group/topic/298005515/,自己在家花200做的大餐,噢力给,165991340,23,12-03 21:10
8,https://www.douban.com/group/topic/298636656/,吃百合吗,牧云放野,77358321,7,12-03 21:05
9,https://www.douban.com/group/topic/298614131/,第一次做冷吃兔,momo,249371779,27,12-03 20:58


# 帖子详情函数

In [4]:
import re
details = pd.DataFrame(columns=['url', 'type', 'text', 'imgs', 'author', 'author_id', 'time', 'location'])

def query_topic(details: pd.DataFrame, url: str,start=0):
    response =requests.request("GET", "{}?start={}".format(url, start), headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    author_data = soup.select("span.from > a")
    author_create_time_data = soup.select("span.create-time.color-green")
    author_location_data = soup.select("span.ip-location")
    author_name, author_id, author_create_time, author_location = '', '', '', ''

    if len(author_data) > 0:
        author_name = author_data[0].text
        author_id = author_data[0]['href'].replace('https://www.douban.com/people/', '').replace('/', '')
    if len(author_create_time_data) > 0:
        author_create_time = author_create_time_data[0].text
    if len(author_location_data) > 0:
        author_location = author_location_data[0].text


    data = soup.select("div.rich-content.topic-richtext > p, div.rich-content.topic-richtext > div")
    # 下面的内容解析了文字也解析了图片，和视频里讲的不一样
    if len(data) > 0:
        for item in data:
            img_tags = item.select("img")
            img_urls = [img_tag["src"] for img_tag in img_tags]
            row = pd.DataFrame({
               'url': url,
               'type': '正文', 
               'text': item.text,
               'imgs': " ".join(img_urls),
               'author': author_name,
               'author_id': author_id,
               'time': author_create_time,
               'location': author_location
            }, index=[0])
            details = pd.concat([details, row], ignore_index=True)
    arr = soup.select("div.reply-doc.content")
    for item in arr:
        img_tags = item.select("img")
        img_urls = [img_tag["src"] for img_tag in img_tags]
        txt = item.select("p.reply-content")[0].text
        author = item.select("h4 > a")[0].text
        author_id = item.select("h4 > a")[0]['href'].replace('https://www.douban.com/people/', '').replace('/', '')
        pubtime = item.select("span.pubtime")[0].text
        pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})\s(.*)"
        match = re.match(pattern, pubtime)
        if match:
            time = match.group(1)
            location = match.group(2)
        row = pd.DataFrame({
               'url': url,
               'type': '评论', 
               'text': txt,
               'imgs': " ".join(img_urls),
               'author': author,
               'author_id': author_id,
               'time': time,
               'location': location
            }, index=[0])
        details = pd.concat([details, row], ignore_index=True)
    if len(arr) >= 100:
        return query_topic(details, url, start+100)
    return details
query_topic(details, "https://www.douban.com/group/topic/288398770/", 0)  

,url,type,text,imgs,author,author_id,time,location
0,https://www.douban.com/group/topic/288398770/,正文,05.13,,水微静,169928054,2023-05-13 12:11:48,江苏
1,https://www.douban.com/group/topic/288398770/,正文,白灼虾，红烧鳊鱼，蔬菜,,水微静,169928054,2023-05-13 12:11:48,江苏
2,https://www.douban.com/group/topic/288398770/,正文,05.14,https://img1.doubanio.com/view/group_topic/l/p...,水微静,169928054,2023-05-13 12:11:48,江苏
3,https://www.douban.com/group/topic/288398770/,正文,05.14,https://img9.doubanio.com/view/group_topic/l/p...,水微静,169928054,2023-05-13 12:11:48,江苏
4,https://www.douban.com/group/topic/288398770/,正文,05.16,https://img3.doubanio.com/view/group_topic/l/p...,水微静,169928054,2023-05-13 12:11:48,江苏
...,...,...,...,...,...,...,...,...
174,https://www.douban.com/group/topic/288398770/,评论,12.01 今晚一人食，炒豌豆苗，牛排，一杯别人投喂的含有桃胶的饮品。,https://img9.doubanio.com/view/richtext/large/...,水微静,169928054,2023-12-01 18:49:23,江苏
175,https://www.douban.com/group/topic/288398770/,评论,12.02 今日一人食：宫保鸡丁，香菇白菜，杂粮饭。,https://img1.doubanio.com/view/richtext/large/...,水微静,169928054,2023-12-02 11:46:26,江苏
176,https://www.douban.com/group/topic/288398770/,评论,12.03 鸡汤，红烧带鱼，糖醋排骨，香菇炒青菜（上得晚了，忘了拍）,https://img9.doubanio.com/view/richtext/large/...,水微静,169928054,2023-12-03 13:26:22,江苏
177,https://www.douban.com/group/topic/288398770/,评论,海鲜出现的频率真的好高，而且感觉做法好多，隔着屏幕都觉得鲜（内陆孩子羡慕哭了T﹏T）,,momo,255078920,2023-12-03 19:51:35,吉林


# Dataframe 与 csv 读写

In [5]:
discussions = pd.concat([query_page(596337, 0),query_page(596337, 30)] , ignore_index=True)
discussions.to_csv('discussions.csv', index=False)

discussions = pd.read_csv('discussions.csv')
for row in discussions.itertuples():
    details = pd.DataFrame(columns=['url', 'type', 'text', 'imgs', 'author', 'time', 'location'])
    details = query_topic(details, row.url, 0)
    pattern = r'/group/topic/(\d+)/'
    topic_id = row.url
    match = re.search(pattern, row.url)
    if match:
        topic_id = match.group(1)
    details.to_csv('topic_{}.csv'.format(topic_id), index=False)

InvalidURL: Failed to parse: 最简单最平凡的幸福莫过于做好了美味的饭菜，和爱的人一起分享，再一起出门散步，撸撸小区的猫咪。?start=0